In [ ]:
class AuthHandler:
    """
    Utils for handling authorization
    """

    log = logging.getLogger(__name__)

    def obtain_id_token(self, client_id: str):
        """
        Obtain OAuth2.0 token to be used with HTTPs requests
        """
        # pylint: disable=import-outside-toplevel
        from google.auth.exceptions import DefaultCredentialsError
        from google.auth.transport.requests import Request
        from google.oauth2 import id_token

        # pylint enable=import-outside-toplevel

        jwt_token = None

        if not client_id:
            self.log.debug(
                "No IAP_CLIENT_ID provided, skipping custom IAP authentication"
            )
            return jwt_token

        try:
            self.log.debug("Attempt to get IAP token for %s", client_id)
            jwt_token = id_token.fetch_id_token(Request(), client_id)
            self.log.info("Obtained JWT token for IAP proxy authentication.")
        except DefaultCredentialsError:
            self.log.warning(
                (
                    " Note that this authentication method does not work with default"
                    " credentials obtained via 'gcloud auth application-default login'"
                    " command. Refer to documentation on how to configure service account"
                    " locally"
                    " (https://cloud.google.com/docs/authentication/production#manually)"
                ),
                exc_info=True,
            )
        except Exception:  # pylint: disable=broad-except
            self.log.error("Failed to obtain IAP access token.", exc_info=True)

        return jwt_token

    def obtain_dex_authservice_session(self, kfp_api):
        """
        Obtain token for DEX-protected service
        """
        if DEX_USERNAME not in os.environ or DEX_PASSWORD not in os.environ:
            self.log.debug("Skipping DEX authentication due to missing env variables")
            return None

        session = requests.Session()
        response = session.get(kfp_api)
        form_relative_url = re.search(
            '/dex/auth/local\\?req=([^"]*)', response.text
        ).group(0)

        kfp_url_parts = urlsplit(kfp_api)
        form_absolute_url = urlunsplit(
            [
                kfp_url_parts.scheme,
                kfp_url_parts.netloc,
                form_relative_url,
                None,
                None,
            ]
        )

        headers = {
            "Content-Type": "application/x-www-form-urlencoded",
        }
        data = {
            "login": os.environ[DEX_USERNAME],
            "password": os.environ[DEX_PASSWORD],
        }

        session.post(form_absolute_url, headers=headers, data=data)
        return session.cookies.get_dict()["authservice_session"]


class MLFlowGoogleOAuthCredentialsProvider(DynamicConfigProvider):
    """
    Uses Google OAuth to generate MLFLOW_TRACKING_TOKEN
    """

    def __init__(self, config: PluginConfig, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.client_id = kwargs["client_id"]

    @property
    def target_config_file(self) -> str:
        return "credentials.yml"

    def generate_config(self) -> dict:
        return {
            "gcp_credentials": {
                "MLFLOW_TRACKING_TOKEN": AuthHandler().obtain_id_token(self.client_id)
            }
        }


class MLFlowGoogleIAMCredentialsProvider(DynamicConfigProvider):
    """
    Uses Google IAM API to generate MLFLOW_TRACKING_TOKEN
    """

    def __init__(self, config: PluginConfig, *args, **kwargs):
        super().__init__(config, *args, **kwargs)
        self.client_id = kwargs["client_id"]
        self.service_account = kwargs["service_account"]

    @property
    def target_config_file(self) -> str:
        return "credentials.yml"

    def generate_config(self) -> dict:
        return {
            "gcp_credentials": {"MLFLOW_TRACKING_TOKEN": self._obtain_credentials()}
        }

    def _obtain_credentials(self):
        from google.cloud import iam_credentials

        client = iam_credentials.IAMCredentialsClient()
        return client.generate_id_token(
            name=f"projects/-/serviceAccounts/{self.service_account}",
            audience=self.client_id,
            include_email=True,
        ).token

In [ ]:
import logging
import os
import re
from urllib.parse import urlsplit, urlunsplit

import requests

from kedro_vertexai.config import PluginConfig
from kedro_vertexai.dynamic_config import DynamicConfigProvider

In [ ]:
aa = AuthHandler()

In [ ]:
aa.obtain_id_token('iap-accessor@gid-advanced-analytics-infra.iam.gserviceaccount.com')

In [ ]:
from google.cloud import iam_credentials

In [ ]:
client = iam_credentials.IAMCredentialsClient()

In [ ]:
client.generate_id_token(
            name=f"projects/-/serviceAccounts/iap-accessor@gid-advanced-analytics-infra.iam.gserviceaccount.com",
            audience="260364966542-721p48q045h4o2hmsmg5dl0gksm7udiu.apps.googleusercontent.com",
            include_email=True,
).token


In [ ]:

        
        return client.generate_id_token(
            name=f"projects/-/serviceAccounts/{self.service_account}",
            audience=self.client_id,
            include_email=True,
        ).token

In [ ]:
import kedro_mlflow

In [ ]:
dir(kedro_mlflow.config.kedro_mlflow_config)

In [ ]:
print(kedro_mlflow.config.kedro_mlflow_config.MlflowParamsOptions.Config)

In [ ]:
dir(kedro_mlflow.config.kedro_mlflow_config.DictParamsOptions)

In [ ]:
import mlflow

In [ ]:
from kedro.framework.context import KedroContext

In [ ]:
dir(KedroContext)

In [ ]:
KedroContext.__dict__

In [ ]:
KedroContext._get_catalog()

In [ ]:
KedroContext["context_helper"].context

In [ ]:
import pandas as pd

In [ ]:
pd.read_parquet('gs://gid-ml-framework-hm-data/03_primary/train_candidates.pq')

In [ ]:
import pandas as pd

In [ ]:
transactions = pd.read_csv('gs://gid-ml-framework-hm-data/01_raw/transactions_train.csv', dtype={'article_id': str})

In [ ]:
transactions.shape

In [ ]:
transactions['t_dat'].min(), transactions['t_dat'].max()

In [ ]:
transactions['t_dat'] = pd.to_datetime(transactions['t_dat'])

In [ ]:
max_date = transactions['t_dat'].max()

In [ ]:
start_train_date = max_date - pd.Timedelta(weeks=6)

In [ ]:
start_train_date

In [ ]:
transactions = transactions[(transactions['t_dat']>=start_train_date)]

In [ ]:
transactions['t_dat'].min(), transactions['t_dat'].max()

In [ ]:
transactions.shape

In [ ]:
transactions.to_csv('gs://gid-ml-framework-hm-data/01_raw/train_transactions_cut.csv')

In [ ]:
from pyarrow.parquet import ParquetFile

In [ ]:
ParquetFile('/Users/mmadej/Downloads/02_intermediate_final_candidates.pq').__dir__()

In [ ]:
ParquetFile('/Users/mmadej/Downloads/02_intermediate_final_candidates.pq').schema

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df = pd.read_csv('~/Downloads/message.csv')

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
reg = LinearRegression(fit_intercept=False).fit(df.x.values.reshape(-1, 1), df.y.values)

In [ ]:
reg.coef_

In [ ]:
reg.__dir__()

In [ ]:
df.y.dtype

In [ ]:
reg


In [ ]:
df.describe()

In [ ]:
df.head(25)

In [ ]:
df['x_lag_1'] = df.x.shift()

In [ ]:
df['y_lag_1'] = df.y.shift()

In [ ]:
df['z'] = df.x+df.y

In [ ]:
(df.x - df.x_lag_1).plot()

In [ ]:
(df.y - df.y_lag_1).plot()

In [ ]:
df.x.plot()

In [ ]:
df.y.plot()

In [ ]:
(df.x+df.y).plot()

In [ ]:
(df.y-df.x).plot()

In [ ]:
(df.x-df.y).plot()

In [ ]:
df

In [ ]:
df['x_delta'] = df.x - df.x_lag_1
df['y_delta'] = df.y - df.y_lag_1

In [ ]:
df

In [ ]:
df[df.x==df.y]

In [ ]:
df['x_sq'] = df.x*df.x
df['y_sq'] = df.y*df.y

In [ ]:
df.describe()

In [ ]:
from gid_ml_framework.image_embeddings.model.pl_autoencoder_module import LitAutoEncoder

In [ ]:
from pytorch_lightning.utilities.parsing import is_picklable

In [ ]:
from gid_ml_framework.image_embeddings.model.pl_encoders import SimpleEncoder
from gid_ml_framework.image_embeddings.model.pl_decoders import SimpleDecoder

In [ ]:
aa = LitAutoEncoder(encoder=SimpleEncoder, decoder=SimpleDecoder)

In [ ]:
aa

In [ ]:
is_picklable(aa)

In [ ]:
LitAutoEncoder

### pyproject.toml to requirements.txt

In [ ]:
import pathlib
import os

In [ ]:
os.chdir('..')

In [ ]:
pathlib.Path.cwd()

In [ ]:
import tomli

In [ ]:
with open("pyproject.toml", mode="rb") as fp:
    config = tomli.load(fp)

In [ ]:
config

In [ ]:
requires = config["tool"]["poetry"]["dependencies"]

In [ ]:
requires

In [ ]:
def _get_version(item):
    if isinstance(item, dict):
        return item.get('version')
    else:
        return item

In [ ]:
requires = [pkg_name+_get_version(pkg_version) for pkg_name, pkg_version in requires.items()]

In [ ]:
requires

In [ ]:
    print(config)
    
    print(requires)
    requires = [pkg_name+pkg_version for pkg_name, pkg_version in requires.items()]